<h1> DattoRMM - Software  </h1>

### Import Core Modules

In [ ]:
# data import and file manipulation
import requests
import json

# response parsing
from lxml import etree
from bs4 import BeautifulSoup
# from requests_html import AsyncHTMLSession
import asyncio

import hvac

#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

### Create report_time var, git location, export location

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

### Import ConfigParser and Create env Variables

In [ ]:
query = [{
    "name":"notepad++",
    "version":"8.5.7"
}]

In [ ]:
def __read_secret__(mount_point="api", path="DATTO_RMM"):

    client = hvac.Client()
    resp = client.kv.v2.read_secret(mount_point=mount_point, path=f'/{path}')
    secret = resp['data']['data']

    return secret

# Create Datto RMM Device DataFrame

In [ ]:
secrets = __read_secret__()

## Create auth token

In [ ]:
config_dict = __read_secret__()

# call token api url
token_uri = f'{config_dict["base_uri"]}/auth/oauth/token'

# construct header
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}


# construct req body
data = {
    'grant_type': 'password',
    'username': secrets['api_key'],
    'password': secrets['api_secret'],
}

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=("public-client", "public"))
content = resp.content.decode("utf-8")
c_dict = json.loads(content)

config_dict["access_token"] = c_dict["access_token"]

## Create DataFrame via API Call Iteration


In [ ]:

## Create Devices DataFrame
# request content response
request_url = f"{config_dict['base_uri']}/api/v2/account/devices"

# construct header
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {config_dict["access_token"]}'
}

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    df_current_page = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_current_page], ignore_index=False)

In [ ]:
df_devices

### Hostname to_upper()

In [ ]:
df_devices['hostname'] = df_devices['hostname'].str.upper()

### Replace Dtypes with Int64

In [ ]:
convert_to_int_mask = ((df_devices.dtypes == 'float') | (df_devices.dtypes == 'bool') | (df_devices.dtypes == 'uint8')) & (df_devices.columns != 'patchStatusPercent')
convert_to_int = df_devices.dtypes[convert_to_int_mask].index.tolist()

In [ ]:
df_devices[convert_to_int] = df_devices[convert_to_int].astype('Int64')

In [ ]:
df_devices = df_devices[['uid','siteName','hostname','intIpAddress','operatingSystem','domain','lastAuditDate','portalUrl','softwareStatus']]

## Filter Devices by siteName in 'datto_rmm_software_management_report.dict'

In [ ]:
df_devices

In [ ]:
df = pd.read_csv('d:/git/example_infrastructure_data_dev/dictionaries/datto_rmm_software_management_report.dict')
software_mgmt = {}
software_mgmt['siteName'] = list(df['siteName'].dropna())
software_mgmt['softwareInstalled'] = list(df['softwareInstalled'].dropna())
software_mgmt['column'] = list(df['column'].dropna())

# Create Software Version DataFrame

## Prepare Data for recall and shaping

## Create Software DataFrame

* Because of the high volume of data from pulling all software and version for each device,
    a method of storing each call response in a JSON document then calling it back keeps memory usage low as objects in memory are created then destroyed in memory but retrained in restful data.
* The same idea is used when calling data out of JSON into a concatenated dataframe after splitting the data up from a nested 'software' dictionary.
* Although this may seem counterintuitive, because each device has different software installed, this method was preferred over using a database because it can be used by anyone.
* If a db or json was not used, there would be an error on each row as columns will never match without the pandas algorithm of the concatenate function 'concat'


### FUNCTION: Explode Software Names / Versions into Columns

In [ ]:
def explode_software(software):
    software_dict = {}
    software_name = software['name']
    software_version = software['version']
    #print(f"Creating dictionary for software: {software_name} with version {software_version}")
    software_dict[software_name] = software_version
    return software_dict

In [ ]:
def software_standard_filter(string):
    for index, e in enumerate(software_mgmt['softwareInstalled']):
        try:
            result = re.findall(e.lower(),string.lower())
            if result:
                print(f'Keyword found: {e}')
                return software_mgmt['column'][index]
                break

        except Exception as e:
            break

In [ ]:
def software_api_req(row):
    # request content response
    request_url = f"{dattormm_config['base_uri']}/api/v2/audit/device/{row['uid']}/software"

    # construct header
    headers = CaseInsensitiveDict()
    headers['Authorization'] = f'Bearer {access_token}'
    headers['Content-Type'] = 'application/json'

    # construct req body
    data = ''

    print(f'\nRequest URL: {request_url}\n\n')

    resp = requests.get(request_url, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    # Create DB Object for Entry
    object_dict = dict(row)

    # object_list.append(object_dict)
    print(object_dict)

    # Explode and Shape Software Dict List Elements
    for software in c_dict['software']:
        software_entry = explode_software(software)
        for k,v in software_entry.items():
            software_name = software_standard_filter(k)
            if software_name:
                object_dict[software_name] = v

    print('*'*50)

    return object_dict

In [ ]:
df_software = pd.DataFrame()

devices_software_list = []

for index, row in df_devices.iterrows():

    devices_software_list.append(software_api_req(row))


df_software = pd.DataFrame(devices_software_list)

# Shape Software DataFrame

### Set Index to device UID

In [ ]:
df_software.set_index('uid', inplace=True)

### FillNA

In [ ]:
df_software.fillna(2,inplace=True)

## Create Boolean columns based on Compliance Version

### Adobe Air

In [ ]:
request_url = f'https://airsdk.harman.com/download'

asession = AsyncHTMLSession()
r = await asession.get(request_url)
await r.html.arender()
resp=r.html.raw_html

soup = BeautifulSoup(resp)
dom = etree.HTML(str(soup))

result_list = []

for node in dom.xpath('//div'):
    text = ''.join(node.itertext()).strip()
    result = re.findall(r'Download\sAdobe\s(\d+\.\d+)',text)
    if result:
        for r in result:
            result_list.append(r)


latest_adobe_air_version = result_list[0]
df_software['latestAdobeAir'] = latest_adobe_air_version
print(f'Latest Adobe Air Version: {latest_adobe_air_version}')

In [ ]:
def is_adobe_air_current(currentVersion):
    if currentVersion != 2:
        if currentVersion == latest_adobe_air_version:
            return 1
        else:
            return 0
    else:
        return 2

In [ ]:
df_software['isAdobeAirCurrent'] = df_software['adobeAir'].apply(is_adobe_air_current)

### 7-Zip

In [ ]:
request_url = f'https://www.7-zip.org/download.html'
resp = requests.get(request_url)
soup = BeautifulSoup(resp.content)
dom = etree.HTML(str(soup))

result_list = []

for node in dom.xpath('//b'):
    text = ''.join(node.itertext()).strip()
    result = re.findall(r'7-Zip\s(\d+\.\d+)',text)
    if result:
        result_list.append(result[0])

latest_7zip_version =  (result_list[0] + '.00.0')
df_software['latest7Zip'] = latest_7zip_version
print(f'Latest 7-Zip Version: {latest_7zip_version}')

In [ ]:
def is_7zip_current(currentVersion):
    if currentVersion != 2:
        if currentVersion == latest_7zip_version:
            return 1
        else:
            return 0
    else:
        return 2

In [ ]:
df_software['is7ZipCurrent'] = df_software['7zip'].apply(is_7zip_current)

### Adobe DC Reader

In [ ]:
request_url = "https://helpx.adobe.com/acrobat/release-note/release-notes-acrobat-reader.html"
resp = requests.get(request_url)

soup = BeautifulSoup(resp.content)
result = re.findall(r'\((\d+\.\d+\.\d+)\)', str(soup))
latest_adobe_dc_reader_version = result[0]
df_software['latestAdobeDCReader'] = latest_adobe_dc_reader_version
print(f'Latest Acrobat Adobe DC Reader Version: {latest_adobe_dc_reader_version}')

In [ ]:
def is_adobe_dc_reader_current(currentVersion):
    if currentVersion != 2:
        if currentVersion == latest_adobe_dc_reader_version:
            return 1
        else:
            return 0
    else:
        return 2

In [ ]:
df_software['isAdobeDCReaderCurrent'] = df_software['adobeAcrobatReaderDC'].apply(is_adobe_dc_reader_current)

### Google Chrome

In [ ]:
product = 'chrome'
platform = 'win64'
channel = 'stable'
version = 'all'

result_list = []

request_url = f"https://versionhistory.googleapis.com/v1/{product}/platforms/{platform}/channels/{channel}/versions/{version}/releases"
resp = requests.get(request_url)
results = re.findall(r'\"version\"\:\s\"(\d+\.\d+\.\d+\.\d+)\"',str(resp.content))

latest_chrome_version = results[0]
df_software['latestChrome'] = latest_chrome_version
print(f'Latest Chrome Version: {latest_chrome_version}')

In [ ]:
def is_chrome_current(currentVersion):
    if currentVersion != 2:
        if currentVersion == latest_chrome_version:
            return 1
        else:
            return 0
    else:
        return 2

In [ ]:
df_software['isChromeCurrent'] = df_software['googleChrome'].apply(is_chrome_current)

### Mozilla Firefox

In [ ]:
request_url = f"[REDACTED]/.json"
resp = requests.get(request_url)
soup = BeautifulSoup(resp.content)
result = re.findall(r'\"LATEST_FIREFOX_VERSION\"\:\s\"(\d+\.\d+\.\d+)\"',str(soup))

latest_firefox_version = result[0]
df_software['latestfirefox'] = latest_firefox_version
print(f'Latest FireFox Version: {latest_firefox_version}')

In [ ]:
def is_firefox_current(currentVersion):
    if currentVersion != 2:
        if currentVersion == latest_firefox_version:
            return 1
        else:
            return 0
    else:
        return 2

In [ ]:
df_software['isFirefoxCurrent'] = df_software['mozillaFirefox'].apply(is_firefox_current)

### MS Teams

In [ ]:
request_url = f"https://docs.microsoft.com/en-us/officeupdates/teams-app-versioning"
resp = requests.get(request_url)
soup = BeautifulSoup(resp.content)

dom = etree.HTML(str(soup))

for node in dom.xpath('/html/body/div[2]/div/section/div/div[1]/main/div[3]/table[4]/tbody/tr[1]/td[3]'):
    text = ''.join(node.itertext()).strip()

latest_teams_version = text
df_software['latestTeams'] = latest_teams_version
print(f'Latest Teams Version: {latest_teams_version}')

In [ ]:
def is_teams_current(currentVersion):
    if currentVersion != 2:
        if currentVersion == latest_teams_version:
            return 1
        else:
            return 0
    else:
        return 2

In [ ]:
df_software['isMSTeamsCurrent'] = df_software['microsoftTeams'].apply(is_teams_current)

### MS Office 365

In [ ]:
request_url = f"https://docs.microsoft.com/en-us/officeupdates/update-history-microsoft365-apps-by-date"
resp = requests.get(request_url)
soup = BeautifulSoup(resp.content)

dom = etree.HTML(str(soup))

for node in dom.xpath('/html/body/div[2]/div/section/div/div[1]/main/div[3]/table[1]/tbody/tr[1]/td[3]'):
    text = ''.join(node.itertext()).strip()

latest_office_365_version = text
df_software['latestOffice365'] = latest_office_365_version
print(f'Latest Office 365 Version: {latest_office_365_version}')

In [ ]:
def is_ms_office_current(currentVersion):
    if currentVersion != 2:
        if currentVersion == latest_office_365_version:
            return 1
        else:
            return 0
    else:
        return 2

In [ ]:
df_software['isMSOfficeCurrent'] = df_software['microsoftOffice365'].apply(is_ms_office_current)

In [ ]:
software_report_cols = [
    'siteName', 'hostname', 'intIpAddress', 'operatingSystem', 'category','domain',
    'lastSeen', 'lastReboot', 'lastAuditDate', 'localTimezone',
    'noAudit7Days', 'offline30Days',
    'portalUrl', 'softwareStatus',
    '7zip','latest7Zip', 'is7ZipCurrent',
    'adobeAir', 'latestAdobeAir', 'isAdobeAirCurrent',
    'adobeAcrobatReaderDC', 'latestAdobeDCReader', 'isAdobeDCReaderCurrent',
    'googleChrome', 'latestChrome',  'isChromeCurrent',
    'mozillaFirefox', 'latestfirefox', 'isFirefoxCurrent',
    'mozillaThunderbird',
    'microsoftOffice365', 'latestOffice365', 'isMSOfficeCurrent',
    'microsoftTeams', 'latestTeams', 'isMSTeamsCurrent',
    'fileZillaClient',
    'java',
    'citrixWorkspace',
]

In [ ]:
df_software = df_software[software_report_cols]

In [ ]:
df_software.replace({2:np.NAN}).to_csv('[REDACTED]/.csv')